# Himmelblau Benchmark
## Imports

In [1]:
from bofire.benchmarks.single import Himmelblau
from bofire.benchmarks.benchmark import run
from bofire.data_models.strategies.api import (
    SoboStrategy,
    RandomStrategy,
    PolytopeSampler,
)
from bofire.data_models.domain.api import Outputs
from bofire.data_models.acquisition_functions.api import qEI
from bofire.data_models.outlier_detection.api import IterativeTrimming, OutlierDetections
from bofire.data_models.surrogates.api import SingleTaskGPSurrogate, MixedSingleTaskGPSurrogate
from bofire.data_models.features.api import CategoricalInput
import bofire.strategies.api as strategies
from bofire.data_models.api import Domain
from functools import partial
import pandas as pd #type: ignore
import os

SMOKE_TEST = os.environ.get("SMOKE_TEST")

## Random Optimization

In [5]:
def sample(domain):
    datamodel = PolytopeSampler(domain=domain)
    sampler = strategies.map(data_model=datamodel)
    sampled = sampler.ask(10)
    return sampled


def best(domain: Domain, experiments: pd.DataFrame) -> float:
    return experiments.y.min()


random_results = run(
    Himmelblau(),
    strategy_factory=RandomStrategy,
    n_iterations=50 if not SMOKE_TEST else 1,
    metric=best,
    initial_sampler=sample,
    n_runs=1,
    n_procs=1,
)

run 00 with current best 4.162: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 59.03it/s]


## SOBO (GPEI) Optimization

In [6]:
bo_results = run(
    Himmelblau(),
    strategy_factory=partial(SoboStrategy, acquisition_function=qEI()),
    n_iterations=50 if not SMOKE_TEST else 1,
    metric=best,
    initial_sampler=sample,
    n_runs=1,
    n_procs=1,
)

run 00 with current best 0.015: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [06:21<00:00,  7.63s/it]


## SOBO (GPEI) Optimization with outlier detection

In [22]:
domain = Himmelblau().domain
detectors = []
for output_feature in domain.outputs.get_keys():
    if len(domain.inputs.get(CategoricalInput, exact=True)):
               detectors.append(IterativeTrimming(
                    base_gp=MixedSingleTaskGPSurrogate(
                        inputs=domain.inputs,
                        outputs=Outputs(features=[domain.outputs.get_by_key(output_feature)]),  # type: ignore
                    ))
                )
    else:
        detectors.append(IterativeTrimming(
            base_gp=SingleTaskGPSurrogate(
                inputs=domain.inputs,
                outputs=Outputs(features=[domain.outputs.get_by_key(output_feature)]),  # type: ignore
            ))
        )
outlier_detection_specs = OutlierDetections(detectors=detectors)

In [23]:
bo_results = run(
    Himmelblau(),
    strategy_factory=partial(SoboStrategy, acquisition_function=qEI(), outlier_detection_specs = outlier_detection_specs,
                             min_experiments_before_outlier_check = 10, 
                            num_experiments_frequency_outlier_check = 2),
    n_iterations=50 if not SMOKE_TEST else 1,
    metric=best,
    initial_sampler=sample,
    n_runs=1,
    n_procs=1,
)

run 00 with current best 0.012: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [06:33<00:00,  7.86s/it]
